In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
import plotly.express as px
import matplotlib.pyplot as plt
import shapely as shp

CENTER_BARCELONA = {"lat": 41.3851, "lon": 2.1734}

df = pd.read_csv(
    "../hospital/opendatabcn_sanitat_hospitals-i-centres-atencio-primaria.csv",
    sep="\t",
    encoding="utf-16",
)
df = df[
    [
        "name",
        "addresses_district_id",
        "addresses_district_name",
        "geo_epgs_4326_lat",
        "geo_epgs_4326_lon",
    ]
]
gdf = gpd.GeoDataFrame(
    df[["name", "addresses_district_id", "addresses_district_name"]],
    geometry=gpd.points_from_xy(df["geo_epgs_4326_lon"], df["geo_epgs_4326_lat"]),
).set_crs(epsg=4326).to_crs(epsg=25831)

gdf_medic = (
    (
        gdf[gdf["name"].str.contains("Hospital") | gdf["name"].str.contains("Clínica")]
        .drop_duplicates("name")
        .reset_index(drop=True)
    )
    .astype(
        {
            "name": "string",
            "addresses_district_id": "int8",
            "addresses_district_name": "category",
        }
    )
    .rename(
        columns={
            "addresses_district_id": "district_code",
            "addresses_district_name": "district_name",
        }
    )
)

In [2]:
df = pd.read_csv("../district_zone/BarcelonaCiutat_Districtes.csv")
gdf_district = (
    gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df["geometria_etrs89"]))
    .drop(columns=["geometria_etrs89", "geometria_wgs84"])
    .rename(
        columns={"Codi_Districte": "district_code", "nom_districte": "district_name"}
    ).astype({"district_code": "int8", "district_name": "category"})
    .set_crs(epsg=25831)
)

mean_distances = {"district_code": [], "mean_distance": []}
for _, row in gdf_district.iterrows():
    minx, miny, maxx, maxy = row["geometry"].bounds

    sum_distances = 0
    counter = 0
    while counter < 10_000:
        point = shp.Point(np.random.random() * (maxx - minx) + minx, np.random.random() * (maxy - miny) + miny)
        if row["geometry"].intersects(point):
            counter += 1
            nearest_hospital = gdf_medic.distance(point).idxmin()
            sum_distances += gdf_medic.distance(point).min()
    
    mean_distances["district_code"].append(row["district_code"])
    mean_distances["mean_distance"].append(sum_distances / counter)

gdf_district = gdf_district.merge(pd.DataFrame(mean_distances), on="district_code").to_crs(epsg=4326).rename(columns={"geometry": "geometry_district"})

gdf_medic = gdf_medic.merge(gdf_district[["district_code", "mean_distance", "geometry_district"]], on="district_code").to_crs(epsg=4326)

In [3]:
gdf_medic.to_csv("mean_distances_hospitals.csv", index=False)

In [34]:
import folium
import branca.colormap as cm

# Create a base map centered around Barcelona
m = folium.Map(location=[CENTER_BARCELONA['lat'], CENTER_BARCELONA['lon']], zoom_start=12, tiles="CartoDB Positron",)

# Create a color map
colormap = cm.linear.viridis.scale(min(mean_distances['mean_distance']), max(mean_distances['mean_distance']))
colormap.caption = 'Distance moyenne des hôpitaux les plus proches (m)'

# Add district polygons to the map
for _, row in gdf_district.iterrows():
    folium.GeoJson(
        row['geometry_district'],
        style_function=lambda _, mean_distance=row['mean_distance']: {
            'fillColor': colormap(mean_distance),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        },
        tooltip=f"{row['district_name']}: {row['mean_distance']:.2f} meters"
    ).add_to(m)

# Add hospital markers to the map
for _, row in gdf_medic.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['name'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Add the color map to the map
colormap.add_to(m)

legend_html = f"""
<div style="position: fixed; 
            top: 10px; left: 50px; 
            border: 1px solid grey; border-radius: 5px; padding: 1px 6px;
            background-color:white;
            z-index:9999; font-size:18px;
            ">
    <b>Distance moyenne d'un habitant à un hôpital pour chaque district</b>
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m.save("barcelona_hospitals_mean_distances.html")